In [4]:
from nltk.stem import RSLPStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk import word_tokenize
import pandas as pd

commentaries = pd.read_csv('../scrapper/commentaries_new.csv')

nltk.download('rslp')

class RSLPTokenizer(object):
    def __init__(self):
        self.rslp = RSLPStemmer()
    def __call__(self, articles):
        return [self.rslp.stem(t) for t in word_tokenize(articles)]

vect = TfidfVectorizer(
    tokenizer = RSLPTokenizer(),
    token_pattern = None,
    lowercase = True,
    strip_accents = 'unicode'
)

vect.fit(commentaries['commentary'])
X = vect.transform(commentaries['commentary'])
X_df = pd.DataFrame(X.toarray(), columns=vect.get_feature_names_out())
X_df.head()

[nltk_data] Downloading package rslp to /home/pedro/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


,!,#,$,%,','','s,(,),*,...,😄,🙌,🙌🏽🙌🏽🙌🏽,🙌🙏😁,🙏,🙏🙌😁,🤣🤣🤣,🤦🏿‍♂️,🥲,🥲descans
0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.789441,0.0,0.0,0.000000,0.0,0.000000,0.135682,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.000000,0.0,0.154413,0.000000,0.197143,0.197143,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.140209,0.0,0.0,0.198629,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# attempted to use BOW instead of TfIdf, but TfIdf is better for accuracy.
# from nltk.corpus import stopwords
# from sklearn.feature_extraction.text import CountVectorizer

# stopwords = stopwords.words('portuguese')
# stopwords.extend(['desses', 'desse'])

# cv = CountVectorizer(
#     stop_words=stopwords,
#     lowercase = True
# )

# counts = cv.fit_transform(commentaries['commentary'])
# X_df = pd.DataFrame(counts.toarray(), columns=cv.get_feature_names_out())
# X_df.head()

# X_df.to_feather('./preprocessed_data/x_bow.feather')

In [2]:
y = commentaries['score']

X_df.to_feather('./preprocessed_data/x_new.feather')
y.to_csv('./preprocessed_data/y_new.csv')